In [1]:
import sys
sys.path.append("..")

from Accuracy_metrics_pipeline import *
from analysis_plots import *

In [2]:
base_gen = "../shuffle_not_allframes"
base_ref = "../../data/palantir_data"
out_base = "../Analysis_results"

proteins = ["3UTQ", "5C0F", "5N1Y", "pep_free"]
fractions = [0.3, 0.4, 0.5]
replicas = [0, 1, 2]


In [3]:

for prot in proteins:
    for frac in fractions:
        frac_tag = int(frac * 100)
        print(f"[RUN] {prot} | fraction={frac}")

        rmsd_refs, rmsd_gens = [], []
        rmsf_refs, rmsf_gens = [], []
        rg_refs, rg_gens = [], []
        X_refs, X_gens = [], []
        cm_refs, cm_gens = [], []
        phi_refs, psi_refs, phi_gens, psi_gens = [], [], [], []

        for rep in replicas:
            gen_xtc = f"{base_gen}/{prot}/{prot}_rep_{rep}/fraction_{frac_tag}/generated_frac_{frac_tag}.xtc"
            ref_xtc = f"{base_ref}/{prot}/{prot}_rep_{rep}_backbone.xtc"
            pdb = f"{base_ref}/{prot}/backbone.pdb"

            gen, ref = load_traj(gen_xtc, ref_xtc, pdb)

            r_ref, r_gen = rmsd_internal(gen, ref)
            rmsd_refs.append(r_ref)
            rmsd_gens.append(r_gen)

            res_ids, rf_ref, rf_gen = rmsf_md_vs_gen(gen, ref)
            rmsf_refs.append(rf_ref)
            rmsf_gens.append(rf_gen)

            rg_ref, rg_gen = radius_of_gyration(gen, ref)
            rg_refs.append(rg_ref)
            rg_gens.append(rg_gen)

            Xr, Xg = pca_projection(gen, ref)
            X_refs.append(Xr)
            X_gens.append(Xg)

            cm_refs.append(compute_contact_map(ref))
            cm_gens.append(compute_contact_map(gen))

            pr, psr, pg, psg = compute_phi_psi(gen, ref)
            phi_refs.append(pr); psi_refs.append(psr)
            phi_gens.append(pg); psi_gens.append(psg)

        out_dir = f"{out_base}/{prot}/fraction_{frac_tag}"

        save(plot_rmsd_panels(rmsd_refs, rmsd_gens), f"{out_dir}/rmsd.png")
        save(plot_rmsf_panels(res_ids, rmsf_refs, rmsf_gens), f"{out_dir}/rmsf.png")
        save(plot_rg_panels(rg_refs, rg_gens), f"{out_dir}/rg.png")
        save(plot_pca_panels(X_refs, X_gens), f"{out_dir}/pca.png")
        save(plot_contact_map_panels(cm_refs, cm_gens), f"{out_dir}/contact_maps.png")
        save(plot_rama_panels(phi_refs, psi_refs, phi_gens, psi_gens), f"{out_dir}/ramachandran.png")

[RUN] 3UTQ | fraction=0.3
[RUN] 3UTQ | fraction=0.4
[RUN] 3UTQ | fraction=0.5
[RUN] 5C0F | fraction=0.3
[RUN] 5C0F | fraction=0.4
[RUN] 5C0F | fraction=0.5
[RUN] 5N1Y | fraction=0.3
[RUN] 5N1Y | fraction=0.4
[RUN] 5N1Y | fraction=0.5
[RUN] pep_free | fraction=0.3
[RUN] pep_free | fraction=0.4
[RUN] pep_free | fraction=0.5
